# 1 准备图数据

import pandas as pd
import math                                                                     
from collections import defaultdict
from collections import Counter
import numpy as np
from tqdm import tqdm
import random
import networkx as nx
import time

road_data_path = '/home/jnli/SHL_2023/SHL2023/road_network/data'
gdf_roads_railway  = nx.read_shp( '{}/England_railwary.shp'.format(road_data_path))
print('rail load well')

gdf_roads_road  = nx.read_shp( '{}/England_road.shp'.format(road_data_path))
print('road load well')

# 看图里面的信息
len(gdf_roads_railway.edges())
for n, data in gdf_roads_railway.nodes(data=True):
    print(n,data)
    break
# # 'key': 标识同一个起点和终点之间不同的路段，即在有向图中，可能存在多个边连接同一对节点，key用于区分它们。
for u, v, data in gdf_roads_railway.edges(data=True):
    # print(data['Wkt'][11:][1:-1].split(','))
    print(u,v,data)
    break

def get_feature(g):
    for u, v, data in g.edges(data=True):
        data_lng_lat = data['Wkt'][11:][1:-1].split(',')
        single_lat_lng = []
        for k in data_lng_lat:
            k = k.strip()
            single_lat_lng.append((float(k.split(' ')[1]),float(k.split(' ')[0])))
        lat_lng.append(single_lat_lng)

        road_code.append(data['code'])
    print('feature load well')

lat_lng = []
road_code = []
get_feature(gdf_roads_railway)
get_feature(gdf_roads_road)

# 构建的是有向图
def create_graph(nodes, node_attrs={}, is_directed=True):
    g = nx.DiGraph()
    # add nodes
    for node in nodes:
        g.add_node(node, **node_attrs.get(node, node_attrs))
    return g
# example usage
nodes = [ i for i in range(len(lat_lng))]
node_attrs = {s_node:{'eid': i+1,'zone_area':((np.min([sv[1] for sv in lat_lng[i]]),np.min([ sv[0] for sv in lat_lng[i]]),np.max([ sv[1] for sv in lat_lng[i]]),np.max([ sv[0] for sv in lat_lng[i]]))),'coords':lat_lng[i],'code':road_code[i]} for i,s_node in enumerate(nodes)}
edge_g = create_graph(nodes, node_attrs=node_attrs)

nx.write_gpickle(edge_g, '{}/graph_shl.gpickle'.format(road_data_path))
edge_g = nx.read_gpickle('{}/graph_shl.gpickle'.format(road_data_path))

roadid2code = {}
for i in range(len(road_code)):
    roadid2code[i+1] = road_code[i]
np.save('{}/roadid2code.npy'.format(road_data_path),roadid2code) # 注意带上后缀名
# Load
print('save well')
roadid2code = np.load('{}/roadid2code.npy'.format(road_data_path),allow_pickle=True).item()

# 2 生成距离差

In [3]:
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import networkx as nx
import math
from collections import Counter
from pre_processing.road_network import load_rn_shp
from pre_processing.map import get_candi_shl

/home/jnli/.conda/envs/RNTrajRec/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


road_data_path = '/home/jnli/SHL_2023/SHL2023/road_network/data'
edge_g = nx.read_gpickle('{}/graph_shl.gpickle'.format(road_data_path))
roadid2code = np.load('{}/roadid2code.npy'.format(road_data_path),allow_pickle=True).item()
# 生成需要的地图
rn = load_rn_shp(edge_g,is_directed=True)# 加载路网数据

print('-----------------------1 get validataion  Hand---------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
feature_data_path = '{}/data_processed_validate_1.csv'.format(data_path)
save_path = data_path# 数据路径
df_point = pd.read_csv(feature_data_path)
get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
print('map data get well')

# print('-----------------------1 get validataion  Hips---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
# feature_data_path = '{}/data_processed_validate_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')

# print('-----------------------1 get validataion  Torso---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
# feature_data_path = '{}/data_processed_validate_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')

# print('-----------------------1 get validataion  Hand---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
# feature_data_path = '{}/data_processed_validate_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')

print('-----------------------2  get train  Bag---------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
feature_data_path = '{}/data_processed_train_1.csv'.format(data_path)
save_path = data_path# 数据路径
df_point = pd.read_csv(feature_data_path)
get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
print('map data get well')

# print('-----------------------2  get train  Hand---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
# feature_data_path = '{}/data_processed_train_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')

# print('-----------------------2  get train  Hips---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
# feature_data_path = '{}/data_processed_train_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')
# print('-----------------------2  get train  Torso---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'
# feature_data_path = '{}/data_processed_train_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')

# print('-----------------------3  get test  ---------------------')
# data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
# feature_data_path = '{}/data_processed_test_1.csv'.format(data_path)
# save_path = data_path# 数据路径
# df_point = pd.read_csv(feature_data_path)
# get_candi_shl(df_point,rn,roadid2code,save_path,search_dis=50)
# print('map data get well')